# **Настройки + библиотеки**

In [1]:
#Установка catboost
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [2]:
!pip install tabgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.7 MB/s eta 0:00:00


In [3]:
# библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tabgan.sampler import OriginalGenerator, GANGenerator, ForestDiffusionGenerator
from catboost import CatBoostClassifier
%matplotlib inline
# подключение диска
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/_ctgan/synthesizer.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Mounted at /content/drive


In [4]:
# датасет
dataset_path = '/content/drive/My Drive/Bases/WIND_DBASE/dag_2011_2022_final.csv'

# интесесущая станция
st_name = 'Кочубей'

# интересующие признаки данных
col_list = ['datetime',
            't_air', 't_soil', 'P_atm', 'wind_dir', 'wind_speed']

# границы обучающей выборки
train_borders = ['2011-01-01 00:00:00', '2020-12-31 21:00:00']
# границы проверочной выборки
val_borders = ['2021-01-01 00:00:00', '2021-12-31 21:00:00']
# границы тестовой выборки
test_borders = ['2022-01-01 00:00:00', '2022-12-31 21:00:00']

# **Функции**

**Общие функции**

In [20]:
# подготовка датасета (чтение, выделение признаков, нормировка, класс ветра)
# path - путь к исходнику
# st_name - название станции
# col_list - колонки признаков
# mode = 'category' / 'values'
def process_dataset(path, st_name, col_list, norm_0_1=False):
  # читаем данные
  df = pd.read_csv(path)
  # выделяем станцию
  df = df[df.st_name == st_name]
  # выделяем признаки
  df = df[col_list]
  # перевод градусов угла направления ветра в индекс лепестка розы ветров [0..7]
  def conv_360_to_8(val):
    if val >= 360:
      return 0
    else:
      return val // 45
    return res
  def conv_360_to_16(val):
    if val >= 360:
      return 0
    else:
      return val // 22.5
    return res
  df['wind_dir'] = df['wind_dir'].apply(conv_360_to_16).astype('int32')

  # округление температуры воздуха
  df['t_air'] = df['t_air'].astype('int32')
  # приведение температуры воздуха к категориям от 0
  df['t_air'] = df['t_air'] - df['t_air'].min()

  # округление температуры почвы
  df['t_soil'] = df['t_soil'].astype('int32')
  # приведение температуры воздуха к категориям от 0
  df['t_soil'] = df['t_soil'] - df['t_soil'].min()

  # округление атмосферного давления
  df['P_atm'] = df['P_atm'].astype('int32')
  # приведение температуры воздуха к категориям от 0
  df['P_atm'] = df['P_atm'] - df['P_atm'].min()

  # приводим скорости ветра к 3 классам:
  # 0 - от 0 до 3 м/с
  # 1 - от 4 до 7 м/с
  # 2 - от 8 м/с и выше
  def wind_to_class(x):
    if x < 4:
      res = 0
    elif x < 8:
      res = 1
    else:
      res = 2
    return res
  df['wind_class'] = df['wind_speed'].apply(wind_to_class).astype('int32')

  if norm_0_1:
    # нормировка столбцов [0..1]
    for col in col_list[1:]:
      a = df[col].min()
      b = df[col].max()
      df[col] = (df[col] - a)/(b - a)
      df[col] = df[col].astype('float32')

  return df

# нарезка примеров из датафрейма
# df - исходный датафрейм
# days - размер блока данных для X примера (по умолчанию 1 день)
# col_list - колонки c которыми делается выборка
# borders - список из двух значений - границ выборки
def create_samples(df, col_list, borders, days_count=1):
  # выделение данных по границам времени
  rab_df = df[(df.datetime >= borders[0]) & (df.datetime <= borders[1])]
  # размер блока данных
  block_size = days_count * 8
  # начальная позиция текущего смещения в датасете
  pos = 0
  # обнуляем списки примеров...
  samples_x = []
  samples_y = []
  # флаг остановки цикла
  Ex = False
  # нарезка данных в цикле
  while not Ex:
    # если не дошли до конца....
    if pos + block_size < rab_df.shape[0]:
      # формирование вектора X для всех колонок
      x_vec = []
      for col in col_list:
        x_vec.extend(rab_df[col][pos:pos+block_size].to_list())
      # формирование вектора Y
      y_vec = rab_df['wind_class'][pos+block_size:pos+block_size+1].to_list()[0]
      # добавление векторов в списки примеров
      samples_x.append(x_vec)
      samples_y.append(y_vec)
    else:
      # прекращение нарезки когда доходим до конца
      Ex = True
    # сдвиг позиции нарезки
    pos += 1 # сдвигаем позицию на 1
  # результат в numpy массивах
  return np.array(samples_x), np.array(samples_y)

# получение весов классов
# df - исходный датафрейм
# borders - список из двух значений - границ выборки
def get_class_weights(df, borders, class_num=3):
  rab_df = df[(df.datetime >= borders[0]) & (df.datetime <= borders[1])]['wind_class']
  class_weights = {}
  for i in range(class_num):
    class_weights[i] = 1.0 / rab_df.value_counts()[i]
  return class_weights

# на основе проверочной или тестовой выборки
# возвращает:
# 1. Точность предсказания
# 2. Список точностей для классов
# 3. Матрицу несоответствий
def get_accuracy_report(model, x, y, class_num=3):
  # предикт
  y_pred = model.predict(x)
  # numpy массив истинных y_true
  y_true = y
  # матрица несоответсвий
  cm = confusion_matrix(y_true, y_pred)
  res = [None for i in range(class_num)]
  for i in range(class_num):
    res[i] = round(cm[i, i]/(cm[i, :].sum()),3)
  return accuracy_score(y_true, y_pred), res, cm


**GAN - Генеративно состязательная сеть для выравнивания классов датасета**

In [6]:
# из numpy массивов тестовой выборки переброс данных в датафрейм
def df_from_np(x, y):
  df = pd.DataFrame(x)
  y1 = []
  for i in range(y.shape[0]):
    y1.append(np.argmax(y[i]))
  df['y'] = y1
  return df

# **Чтение и обработка данных, подготовка примеров**

In [21]:
#загрузка и предобработка данных
df = process_dataset(dataset_path, st_name, col_list)
df

,datetime,t_air,t_soil,P_atm,wind_dir,wind_speed,wind_class
2,2011-01-01 00:00:00,33,34,26,15,3,0
6,2011-01-01 03:00:00,33,33,25,15,3,0
9,2011-01-01 06:00:00,33,33,27,13,3,0
14,2011-01-01 09:00:00,32,34,27,13,4,1
17,2011-01-01 12:00:00,31,32,27,13,3,0
...,...,...,...,...,...,...,...
139988,2022-12-31 09:00:00,33,38,38,0,0,0
139993,2022-12-31 12:00:00,36,37,37,6,1,0
139998,2022-12-31 15:00:00,32,28,37,0,0,0
140000,2022-12-31 18:00:00,29,28,42,0,0,0


In [22]:
# количество дней в блоке данных
days_count = 1
# нарезка примеров для обучения
train_x, train_y = create_samples(df, col_list[1:], train_borders, days_count=days_count)
# нарезка примеров для проверки
val_x, val_y = create_samples(df, col_list[1:], val_borders, days_count=days_count)
# нарезка примеров для теста
test_x, test_y = create_samples(df, col_list[1:], test_borders, days_count=days_count)
# размеры выборки
print('Обучающая:', train_x.shape,  train_y.shape)
print('Проверочная:', val_x.shape,  val_y.shape)
print('Тестовая:', test_x.shape,  test_y.shape)
# настройка весов классов для обучающей выборки
class_weight = get_class_weights(df, train_borders)
print(class_weight)

Обучающая: (29216, 40) (29216,)
Проверочная: (2912, 40) (2912,)
Тестовая: (2912, 40) (2912,)
{0: 6.947821857847565e-05, 1: 8.482483671218933e-05, 2: 0.0003287310979618672}


**Генерация выборки GAN**

In [23]:
# переброс данных из обучающих примеров в датафрейм
df = df_from_np(train_x, train_y)
df['y'].value_counts()

0    29216
Name: y, dtype: int64

In [ ]:
# переброс данных из проверочных примеров в датафрейм
#df_test = df_from_np(val_x, val_y)
#df_test['y'].value_counts()

In [ ]:
# берем с последнего класса 3042 значения, а с остальних семплируем такое же количество примеров
#df0 = pd.concat([df[df['y']==2], df[df['y']==0].sample(len(df[df['y']==2])),
#                df[df['y']==1].sample(len(df[df['y']==2]))])
# перемешиваем
#df0 = df0.sample(frac=1)
#df0['y'].value_counts()

In [ ]:
# генерируем выборку через GAN
df_gan1, y1 = OriginalGenerator(gen_x_times=10).generate_data_pipe(df.drop(['y'], axis=1), df[['y']], df.drop(['y'], axis=1), )
df_gan2, y2 = GANGenerator(gen_x_times=10).generate_data_pipe(df.drop(['y'], axis=1), df[['y']], df.drop(['y'], axis=1), )
df_gan1['y'] = y1
df_gan2['y'] = y2
df_gan = pd.concat([df_gan1, df_gan2])

In [ ]:
df_gan['y'].value_counts()

0    91191
1    64396
2    20032
Name: y, dtype: int64

In [24]:
#gan_path = '/content/drive/My Drive/Bases/WIND_DBASE/kochubey_train_gan_1.csv'
#df_gan.to_csv(gan_path, index=False)
df_gan = pd.read_csv('/content/drive/My Drive/Bases/WIND_DBASE/kochubey_train_gan_1.csv')

In [ ]:
# группируем данные 0 и 1 классы из старой выборки, 2 класс из сгенерированной
#df_new = pd.concat([df[df['y']<2], df_gan[df_gan['y']==2]])
# перемешиваем
#df_new = df_new.sample(frac=1)
#df_new['y'].value_counts()

In [25]:
# формируем новые массивы для обучения
train_x_new = np.array(df_gan.drop(['y'], axis=1))
train_y_new = np.array(df_gan['y'].to_list())
print(train_x_new.shape)
print(train_y_new.shape)

(175619, 40)
(175619,)


In [26]:
class_weight = {}
for i in range(3):
  class_weight[i] = 1.0 / df_gan['y'].value_counts()[i]

# **ОБУЧЕНИЕ МОДЕЛЕЙ**

In [18]:
# Объявление CatBoostClassifier и обучение
model = CatBoostClassifier(iterations = 1000, class_weights=class_weight)
model.fit(train_x_new, train_y_new)

Learning rate set to 0.10299
0:	learn: 1.0697365	total: 187ms	remaining: 3m 6s
1:	learn: 1.0468270	total: 314ms	remaining: 2m 36s
2:	learn: 1.0274969	total: 438ms	remaining: 2m 25s
3:	learn: 1.0122661	total: 574ms	remaining: 2m 22s
4:	learn: 0.9990960	total: 714ms	remaining: 2m 22s
5:	learn: 0.9880267	total: 841ms	remaining: 2m 19s
6:	learn: 0.9761101	total: 970ms	remaining: 2m 17s
7:	learn: 0.9668435	total: 1.09s	remaining: 2m 15s
8:	learn: 0.9589004	total: 1.24s	remaining: 2m 16s
9:	learn: 0.9499541	total: 1.36s	remaining: 2m 14s
10:	learn: 0.9421180	total: 1.5s	remaining: 2m 14s
11:	learn: 0.9368428	total: 1.63s	remaining: 2m 14s
12:	learn: 0.9310586	total: 1.76s	remaining: 2m 13s
13:	learn: 0.9258913	total: 1.88s	remaining: 2m 12s
14:	learn: 0.9201791	total: 2.01s	remaining: 2m 11s
15:	learn: 0.9161169	total: 2.14s	remaining: 2m 11s
16:	learn: 0.9123655	total: 2.26s	remaining: 2m 10s
17:	learn: 0.9085409	total: 2.38s	remaining: 2m 9s
18:	learn: 0.9046003	total: 2.5s	remaining: 2m 9

In [27]:
# Проверочная выборка
total_accuracy, accuracies, cm = get_accuracy_report(model, val_x, val_y)
for index, value in enumerate(accuracies):
  print(f"Точность класса {index}: {round(value*100, 2)} %")
print(cm)
print(f"Общая точность: {total_accuracy}")

Точность класса 0: 77.2 %
Точность класса 1: 65.6 %
Точность класса 2: 67.1 %
[[1176  326   21]
 [ 266  694   98]
 [  18   91  222]]
Общая точность: 0.7184065934065934


In [28]:
# Тестовая выборка
total_accuracy, accuracies, cm = get_accuracy_report(model, test_x, test_y)
for index, value in enumerate(accuracies):
  print(f"Точность класса {index}: {round(value*100, 2)} %")
print(cm)
print(f"Общая точность: {total_accuracy}")

Точность класса 0: 88.7 %
Точность класса 1: 67.7 %
Точность класса 2: 50.0 %
[[2256  285    1]
 [  97  237   16]
 [   1    9   10]]
Общая точность: 0.8595467032967034
